# **Modelling and Tuning**

In [1]:
import pandas as pd
import numpy as np

In [2]:
import warnings
from sklearn.exceptions import ConvergenceWarning

# Ignore ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

In [3]:
import pickle

# Load train/val/test split data from notebook3
with open(r'Nata_Files\\train_test_split.pkl', 'rb') as f:
    notebook3_data = pickle.load(f)

# Core datasets
X = notebook3_data.get('X')
y = notebook3_data.get('y')

# Splits and processed feature sets
X_train = notebook3_data.get('X_train')
y_train = notebook3_data.get('y_train')
X_val = notebook3_data.get('X_val')
y_val = notebook3_data.get('y_val')
X_test = notebook3_data.get('X_test')
y_test = notebook3_data.get('y_test')
X_train_val = notebook3_data.get('X_train_val')
y_train_val = notebook3_data.get('y_train_val')

numeric_cols = notebook3_data.get('numeric_cols')
kf = notebook3_data.get('kf')
rkf = notebook3_data.get('rkf')
skf = notebook3_data.get('skf')

print("Train/Val/Test split data loaded successfully!")
if X is not None and y is not None:
    print(f"Full dataset X shape: {X.shape} | y shape: {y.shape}")
if X_train is not None:
    print(f"X_train shape: {X_train.shape}")
if X_val is not None:
    print(f"X_val shape: {X_val.shape}")
if X_test is not None:
    print(f"X_test shape: {X_test.shape}")
if kf is not None:
    try:
        print(f"kf splits: {kf.get_n_splits()}")
    except Exception:
        print("kf loaded (object), get_n_splits() unavailable for this object")
if rkf is not None:
    try:
        print(f"rkf splits: {rkf.get_n_splits()}")
    except Exception:
        print("rkf loaded (object), get_n_splits() unavailable for this object")

Train/Val/Test split data loaded successfully!
Full dataset X shape: (5196, 14) | y shape: (5196,)
X_train shape: (3117, 14)
X_val shape: (1039, 14)
X_test shape: (1040, 14)
kf splits: 10
rkf splits: 14


In [4]:
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedKFold


In [5]:
class Predictor:
    def __init__(self, model):
        self.model = model

    def fit(self, X, y):
        self.model.fit(X, y)

    def predict_proba(self, X_val):
        return self.model.predict_proba(X_val)
    
    def predict(self, X_val):
        return self.model.predict(X_val)

In [6]:
def avg_score_clf(method,X,y,model):
    score_train = []
    score_val = []
    for train_index, val_index in method.split(X):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]
        trained_model = fit_model(model, X_train, y_train)
        value_train = eval_clf_model(trained_model, X_train, y_train)
        value_val = eval_clf_model(trained_model, X_val, y_val)
        score_train.append(value_train)
        score_val.append(value_val)

    print('Train:', np.mean(score_train))
    print('Validation:', np.mean(score_val))

In [7]:
def avg_score_clf_skf(method, X, y, model):
    score_train = []
    score_val = []
    for train_index, val_index in method.split(X, y):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]
        trained_model = fit_model(model, X_train, y_train)
        value_train = eval_clf_model(trained_model, X_train, y_train)
        value_val = eval_clf_model(trained_model, X_val, y_val)
        score_train.append(value_train)
        score_val.append(value_val)

    print('Train:', np.mean(score_train))
    print('Validation:', np.mean(score_val))

## **Model Selection**

Que modelos podemos usar?
Isto é um classification problem então podemos descartar logo alguns.
Podemos testar :
- Logistic Regression
- Decision Tree Classifier
- Random Forest
- K nearest Classifier
- stacking /ensemble

## **Model Evaluation**

In [8]:
logr = Predictor(LogisticRegression())
logr.fit(X_train, y_train)
logr_proba = logr.predict_proba(X_val)[:,1]
logr_pred = logr.predict(X_val)


dtc = Predictor(DecisionTreeClassifier(max_depth= 5))
dtc.fit(X_train, y_train)
dtc_proba = dtc.predict_proba(X_val)[:,1]
dtc_pred = dtc.predict(X_val)


rf = Predictor(RandomForestClassifier())
rf.fit(X_train, y_train)
rf_proba = rf.predict_proba(X_val)[:,1]
rf_pred = rf.predict(X_val)


knn_clf = Predictor(KNeighborsClassifier())
knn_clf.fit(X_train, y_train)
knn_clf_proba = knn_clf.predict_proba(X_val)[:,1]
knn_clf_pred = knn_clf.predict(X_val)


#### **Logistic Regression**

##### Stratified K-Fold CV

In [9]:
avg_score_clf_skf(skf, X, y,logr)

NameError: name 'fit_model' is not defined

#### **Decision Tree Classifier**

In [ ]:
dtc = Predictor(DecisionTreeClassifier(max_depth= 5))
dtc.fit(X_train, y_train)
dtc.predict_proba(X_val)

array([[0.4375    , 0.5625    ],
       [0.05050505, 0.94949495],
       [0.52272727, 0.47727273],
       ...,
       [0.30319149, 0.69680851],
       [0.80597015, 0.19402985],
       [0.79227053, 0.20772947]])

##### Stratified K-Fold CV

In [ ]:
avg_score_clf_skf(skf, X, y, dtc)

NameError: name 'fit_model' is not defined

#### **Random Forest**

In [ ]:
rfc = RandomForestClassifier()

In [ ]:
avg_score_clf_skf(skf, X, y, rfc)

NameError: name 'fit_model' is not defined

#### **K-Neighbors Classifier**

In [ ]:
knn_clf = KNeighborsClassifier()

##### Stratified K-Fold CV

In [ ]:
avg_score_clf_skf(skf, X, y, knn_clf)

NameError: name 'fit_model' is not defined

In [ ]:
def get_metrics(y_val, y_proba, y_pred, model):
    return {
        "Model" : model,
        "AUC": roc_auc_score(y_val, y_proba),
        "Accuracy": accuracy_score(y_val, y_pred),
    }

In [ ]:
models_metrics = []

models_metrics.append(get_metrics(y_val, logr_proba, logr_pred, "Logistic Regression"))
models_metrics.append(get_metrics(y_val, dtc_proba, dtc_pred, "DTClassifier"))
models_metrics.append(get_metrics(y_val, rf_proba, rf_pred, "Random Forest"))
models_metrics.append(get_metrics(y_val, knn_clf_proba, knn_clf_pred, "KNClassifier"))

In [ ]:
df_models_metrics = pd.DataFrame(models_metrics)
df_models_metrics.set_index("Model", inplace=True)

df_models_metrics

,AUC,Accuracy
Model,,
Logistic Regression,0.809067,0.750722
DTClassifier,0.768210,0.728585
Random Forest,0.840843,0.767084
KNClassifier,0.774752,0.726660


Fazer ensemble (bagging)?
Perguntar ao professor se podemos usar só RF? Vale a pena testar ensemble dps? ir testando
A data partitioning aqui está aser hold-out method, gostava de testar Stratified CV da mesma maneira

FAZER:
- Tune do modelo
- Hyperparameters?
- Feature selection (if appropriate)